In [9]:
import os
import sqlite3
import logging
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import requests

In [7]:
google_json = "D:/coding/instagram/scripts/private/insta-401020-8a55316147d7.json"
img_path = "D:/coding/instagram/scripts/downloaded_images/image_0.jpg"

credentials = Credentials.from_service_account_file(
    google_json,
    scopes=['https://www.googleapis.com/auth/drive']
)
drive_service = build('drive', 'v3', credentials=credentials)

file_metadata = {'name': os.path.basename(img_path)}
media = MediaFileUpload(img_path, mimetype='image/jpeg')
file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

file_id = file.get('id')

# make the file publicly accessible and retrieve sharing link
file_id = file.get('id')
drive_service.permissions().create(
    fileId=file_id,
    body={'type': 'anyone', 'role': 'reader'},
    fields='id'
).execute()
public_url = f"https://drive.google.com/uc?export=view&id={file_id}"

print(f"Public URL: {public_url} has been created")

Public URL: https://drive.google.com/uc?export=view&id=1lcCUfowNnOCoR_ESI7__qCGRRyxA4vsW has been created


In [11]:

insta_token = 'EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD'
insta_user_id = '17841461285196685'
url = f"https://graph.facebook.com/v18.0/17841461285196685/media"
# create container
params = {
    'image_url': public_url,
    'caption': 'e',
    'access_token': insta_token
}

response = requests.post(url, params=params)
print(response.json())
# grabs media object id
container_id = response.json().get('id', 'ID not found')
#publish pic
publish_url = f"https://graph.facebook.com/v18.0/{insta_user_id}/media_publish?creation_id={container_id}&access_token={insta_token}"

response = requests.post(publish_url)

print(f'post has been created: {publish_url}')





{'id': '18239874154229539'}
post has been created: https://graph.facebook.com/v18.0/17841461285196685/media_publish?creation_id=18239874154229539&access_token=EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD


In [13]:
class PostImg:

    @staticmethod
    def setup_logging():
        logging.basicConfig(filename='post_operations.log', level=logging.INFO)

    def __init__(self, google_json, insta_access_token, insta_user_id, default_hashtags, db_path):
        self.google_json = google_json
        self.insta_access_token = insta_access_token
        self.insta_user_id = insta_user_id
        self.default_hashtags = default_hashtags
        self.db_path = db_path
        self.ids = []
        self.top_post = None
        self.owner_id = None
        self.hashtags = None
        self.public_url = ()
        self.owner_username = ()


    def connect_db(self):
        try:
            conn = sqlite3.connect(self.db_path)
            print('db connected')
            return conn
        except sqlite3.Error as e:
            logging.error(f"db error: {e}")
            return None    
        
    def get_posted_posts(self):
        ## get ids that have been posted to cross reference
        try:
            with open('posted_ids', 'r') as f:
                self.ids = [line.strip() for line in f]
        except FileNotFoundError:
            logging.warning("posted_ids file not found")

    def get_top_post(self):
        print('get_top_post function running')
        conn = self.connect_db()
        if conn:
            cur = conn.cursor()
            cur.execute("SELECT MAX(likesCount), id, hashtags, ownerId FROM insta_hashtag")
            row = cur.fetchone()
            #loop till post has matching downlaoded img
            
            while row:
                self.top_post, self.post_id, self.hashtags, self.owner_id = row
                
                img_path = os.path.join('downloaded_images', f'image_{self.post_id}.jpg')
                
                if os.path.exists(img_path):
                    break  
            
                # If image doesn't exist, fetch next row with lesser likesCount
                cur.execute("SELECT MAX(likesCount), id, hashtags, ownerId FROM insta_hashtag WHERE likesCount < ?", (self.post_id))
                row = cur.fetchone()
               
            if row is None:
                print("No more rows to fetch.")
                logging.info('No more rows to fetch')


    def get_img(self):
        self.img_path = os.path.join('downloaded_images', f'image_{self.post_id}.jpg')
        
        if os.path.exists(self.img_path):
            logging.info('Matching image fetched')
            print('Matching image fetched')
        else:
            logging.warning('Image not found')
            print('Image not found')
 


    def google_drive(self):
        try:
            credentials = Credentials.from_service_account_file(
                self.google_json,
                scopes=['https://www.googleapis.com/auth/drive']
            )
            drive_service = build('drive', 'v3', credentials=credentials)

            file_metadata = {'name': os.path.basename(self.img_path)}
            media = MediaFileUpload(self.img_path, mimetype='image/jpeg')
            file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

            file_id = file.get('id')

            # make the file publicly accessible and retrieve sharing link
            file_id = file.get('id')
            drive_service.permissions().create(
                fileId=file_id,
                body={'type': 'anyone', 'role': 'reader'},
                fields='id'
            ).execute()
            self.public_url = f"https://drive.google.com/uc?export=view&id={file_id}"

            logging.info(f"Public URL: {self.public_url} has been created")
            print(f"Public URL: {self.public_url} has been created")

        except Exception as e:
            logging.error(f"Google Drive error: {e}")


    def get_owner_username(self):
        #get username of the author
        try:
            owner_url = f"https://graph.instagram.com/{self.owner_id}?fields=username&access_token={self.insta_access_token}"
            owner_response = requests.get(owner_url)
            owner_data = owner_response.json()
            self.owner_username = owner_data.get('username', 'Username not found')
            logging.info(f'Owner username acquired: {self.owner_username}')
            print(f'Owner username acquired: {self.owner_username}')

        except requests.RequestException as e:
            logging.error(f"Request error: {e}")

    def generate_caption(self):
        #get hashtags from post or if none default
        if self.hashtags == None:
            self.hashtags == self.default_hashtags
            print('post has no hastags')
        
        self.caption = f'''⅋ 🪡

                        Via. {self.owner_username}

                        —————————————————-
                        𝘍𝘰𝘭𝘭𝘰𝘸 @clavext 𝘧𝘰𝘳 𝘮𝘰𝘳𝘦 𝘤𝘰𝘯𝘵𝘦𝘯𝘵
                        .
                        .
                        .
                        .
                        .
                        .
                        .
                        .
                        {self.hashtags}'''
        

    def insta_api_post(self):
        url = f"https://graph.facebook.com/v18.0/{self.insta_user_id}/media"
        # create container
        params = {
            'image_url': self.public_url,
            'caption': self.caption,
            'access_token': self.insta_access_token
        }
        try:
            response = requests.post(url, params=params)
            print(response.json())
            # grabs media object id
            container_id = response.json().get('id', 'ID not found')
            #publish pic
            publish_url = f"https://graph.facebook.com/v18.0/{self.insta_user_id}/media_publish?creation_id={container_id}&access_token={self.insta_access_token}"

            response = requests.post(publish_url)
            logging.info(response.json())
            print(f'post has been created: {publish_url}')
            # create a file that add posted ids into it
            with open('posted_ids', 'a') as f:
                f.write(str(self.top_post) + '\n')

        except requests.RequestException as e:
            logging.error(f"Request error: {e}")

 
config = {
    "google_json": "D:/coding/instagram/scripts/private/insta-401020-8a55316147d7.json",
    "insta_access_token": 'EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD',
    "insta_user_id": '17841461285196685',
    "default_hashtags": ["gorpcore","outerwear", "gorp", "gorpcorefashion", "outdoors", 
                         "arcteryx", "salomon", "gorpcorefashion", "gorpcorestyle", "functionalarchive", 
                         "ootd", "explore", "getoutside"],
    "db_path": "D:/coding/instagram/scripts/insta_hashtag.db"
}
if __name__ == "__main__":
    PostImg.setup_logging()
    post = PostImg(**config)
    post.get_posted_posts()
    post.get_top_post()
    post.get_img()
    post.google_drive()
    post.get_owner_username()
    # post.generate_caption()
    # post.insta_api_post()
    

get_top_post function running
db connected


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 19 supplied.

In [15]:
try:
    owner_id = '38758293043'
    insta_token
    owner_url = f"https://graph.instagram.com/{owner_id}?fields=username&access_token={insta_token}"
    owner_response = requests.get(owner_url)
    owner_response.raise_for_status()
    owner_data = owner_response.json()
    owner_username = owner_data.get('username', 'Username not found')
    print(f'Owner username acquired: {owner_username}')
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


An error occurred: 400 Client Error: Bad Request for url: https://graph.instagram.com/38758293043?fields=username&access_token=EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD


In [28]:
import requests

try:
    insta_token = 'EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD'
    owner_id = '3082218057618335681'
    owner_url = f"https://graph.instagram.com/{owner_id}?fields=username&access_token={insta_token}"
    owner_response = requests.get(owner_url)
    owner_response.raise_for_status()
    owner_data = owner_response.json()
    owner_username = owner_data.get('username', 'Username not found')
    print(f'Owner username acquired: {owner_username}')
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


An error occurred: 400 Client Error: Bad Request for url: https://graph.instagram.com/3082218057618335681?fields=username&access_token=EAAPRAXRSsOQBOZB22Mf0mrWwvB0MIJlH2qMVWKWe62A3DqRVw5kcvqhtlSvn7hTepFZCGevAyZC0FmaTJI1i34E16h9u5haXnsA6VedCVivLGrQFyX6Kp4wGga1iNL7h8daHrcPedIctk5l3PeUXfqKiNtOQYBzeKw1ft2ZA5cE7q1y68LCFW2hPHW5rZBFS33KcoWNqdWtqu6tyotL71rJ3Rm3pXptbOCIXIO5Tk9UYZD


In [34]:
from urllib.parse import urlparse

def get_username_from_url(url):
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    if len(path_parts) >= 2:
        return path_parts[2]
    else:
        return "Username not found"

# Test the function
url = "https://www.instagram.com/p/CrGPP48oqfB/"
username = get_username_from_url(url)
print(f"Username: {username}")

Username: CrGPP48oqfB


In [29]:
import instaloader

# Initialize Instaloader object
L = instaloader.Instaloader()

# Define the shortcode of the post
shortcode = 'CrGPP48oqfB'  # Replace with your actual shortcode

# Get the Post object
post = instaloader.Post.from_shortcode(L.context, shortcode)

# Access the owner's username
owner_username = post.owner_username

# Print the username
print(f"Post owner's username: {owner_username}")


Post owner's username: hyoh456


In [38]:
hashtags= ["ootd", "arctery\u00d7", "outdoor", "outdoors", "explore", "gorpcore", "goretexstudio", "hypebeast", "grailed", "fashionmoodboard", "outdoorlife", "hiking", "itsbetteroutside", "outdoorism", "getoutside", "gorpworld"]
hashtags = [word.strip().replace('"', '') for word in hashtags]
hashtags_with_hash = ['#' + tag for tag in hashtags]
result = ' '.join(hashtags_with_hash)
print(result)

#ootd #arctery× #outdoor #outdoors #explore #gorpcore #goretexstudio #hypebeast #grailed #fashionmoodboard #outdoorlife #hiking #itsbetteroutside #outdoorism #getoutside #gorpworld


In [39]:
import sys
print(sys.executable)


C:\Users\User\AppData\Local\Programs\Python\Python311\python.exe
